In [62]:
import pandas as pd
import numpy as np
import math

In [79]:
nls_df = pd.read_csv('national_longitudinal_survey_of_youth_1979.csv')

# preprocessing
nls_df = nls_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
nls_df.loc[nls_df['SEX OF R']<= 0, 'SEX OF R'] = None
nls_df.loc[nls_df['SEX OF R']== 1, 'SEX OF R'] = "Male"
nls_df.loc[nls_df['SEX OF R']==2, 'SEX OF R'] = "Female"

# preselection 
nls_df = nls_df[nls_df['SEX OF R']=="Female"]  #we're only interested in the women (because they're the ones giving birth)

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [96]:
# populate column gave_birth_to_child
# also: interpolation for NUMBER OF BIOLOGICAL CHILDREN REPORTED

nls_df['gave_birth_to_child'] = 0
nls_df = nls_df.sort_values(['IDENTIFICATION CODE','year'])
nls_df.index = range(len(nls_df))
previous_number_of_children = {id_code: np.nan for id_code in  list(nls_df['IDENTIFICATION CODE'].unique())}


for index, row in nls_df.iterrows():
    if index%1000 ==0:
        print(index)
    id_code = row['IDENTIFICATION CODE']
    if math.isnan(row['NUMBER OF BIOLOGICAL CHILDREN REPORTED']):
        nls_df.loc[index,'NUMBER OF BIOLOGICAL CHILDREN REPORTED'] = previous_number_of_children[id_code]
    else:
        nls_df.loc[index,'gave_birth_to_child'] = row['NUMBER OF BIOLOGICAL CHILDREN REPORTED'] - previous_number_of_children[id_code]
        previous_number_of_children[id_code] = row['NUMBER OF BIOLOGICAL CHILDREN REPORTED']
        
# let's only accept values that are 0 or 1 (this is the majority - see next row)        
# nls_df.groupby('gave_birth_to_child').aggregate({'gave_birth_to_child':'first','IDENTIFICATION CODE':'count'})
nls_df.loc[~nls_df['gave_birth_to_child'].isin([0,1]),'gave_birth_to_child'] = 0

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [124]:
# remove entries with missing values
nls_df = nls_df[~nls_df['NUMBER OF BIOLOGICAL CHILDREN REPORTED'].isna()]
nls_df = nls_df[~nls_df['MARITAL STATUS'].isna()]
nls_df = nls_df[~nls_df['AGE AT INTERVIEW DATE'].isna()]

# create features
# has 1 child
nls_df['has 1 child'] = 0
nls_df.loc[nls_df['NUMBER OF BIOLOGICAL CHILDREN REPORTED']==1,'has 1 child'] = 1

# has 2 children
nls_df['has 2 children'] = 0
nls_df.loc[nls_df['NUMBER OF BIOLOGICAL CHILDREN REPORTED']==2,'has 2 children'] = 1

# has more than 2 children
nls_df['has more than 2 children'] = 0
nls_df.loc[nls_df['NUMBER OF BIOLOGICAL CHILDREN REPORTED']>2,'has more than 2 children'] = 1

# is_married
nls_df['is_married'] = 0
nls_df.loc[nls_df['MARITAL STATUS']=='Married','is_married'] = 1

# age^2
nls_df['age_squared'] = nls_df['AGE AT INTERVIEW DATE'] * nls_df['AGE AT INTERVIEW DATE']

In [164]:
nls_df = nls_df[['has 1 child','has 2 children','has more than 2 children','is_married', 'AGE AT INTERVIEW DATE','age_squared', 'gave_birth_to_child']]
nls_df.to_csv('dataset_for_logistic_regression.csv')

In [172]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X = nls_df[['has 1 child','has 2 children','has more than 2 children','is_married', 'AGE AT INTERVIEW DATE','age_squared']]
X['y_intercept'] = 1
y = nls_df['gave_birth_to_child']

X_train = X[:50000]
y_train = y[:50000]
clf = LogisticRegression(random_state=0,fit_intercept=False).fit(X_train, y_train)


X_test = X[50001:]
y_test = y[50001:]
clf.score(X_test, y_test)

c:\programdata\miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.905927948559852

In [173]:
print(clf.coef_)

[[-3.37198642e+00  1.31086114e+00  1.73538964e+00  4.75135455e-01
  -8.48595893e-02 -6.42634684e-04 -3.25735649e-01]]


In [169]:
[[-3.78200691  3.32004304  3.81736928  0.2896024  -0.46164898  0.00605224]]


SyntaxError: invalid syntax (<ipython-input-169-b287f4e3901a>, line 1)

In [174]:
x_y_values_for_linear_regression = pd.read_csv('C:/Users/l412/Documents/1 projects/2020-05-21 Lifecycle Simulation/data/National Longitudinal Survey 1979/x-y-values-for-learning-birth-probaility.csv')

# small correction concerning Alter2
x_y_values_for_linear_regression['Alter2'] = x_y_values_for_linear_regression['Alter'] * x_y_values_for_linear_regression['Alter']

In [175]:
X = x_y_values_for_linear_regression[['schon_1_Kind', 'schon_2_Kinder', 'schon_mehr_als_2_Kinder', 'verheiratet', 'Alter', 'Alter2']]
y = x_y_values_for_linear_regression['child_increase']

In [176]:
X['y_intercept'] = 1

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [177]:
X_train = X[:35000]
y_train = y[:35000]
clf = LogisticRegression(random_state=0,fit_intercept=False).fit(X_train, y_train)


X_test = X[35001:]
y_test = y[35001:]
clf.score(X_test, y_test)

c:\programdata\miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8447080573101217

In [178]:
print(clf.coef_)

[[-3.78200691  3.32004304  3.81736928  0.2896024  -0.46164898  0.00605224
   3.35540541]]


In [ ]:
R:
0.890214939 -18.767619926  -0.479066890   0.156509189  -0.061962232